In [1]:
# !pip install pandas
# !pip install openpyxl

In [2]:
import pandas as pd
import glob

## 최신 법정동 코드 DF

- 법정동명이 '서울특별시 OO구'인 것만 남기고, 'OO구'로 변환
- 딕셔너리로 생성 : {'법정동명':법정동코드}

In [3]:
new_code_df = pd.read_excel('../dedev_2nd_project/법정동코드 조회자료.xlsx')
new_code_df.head()

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동
3,1111010200,서울특별시 종로구 신교동
4,1111010300,서울특별시 종로구 궁정동


In [4]:
filtered_gucode_df = new_code_df[new_code_df['법정동명'].str.endswith('구')]
new_code_dic = {row['법정동명'].split()[-1]: row['법정동코드'] for _, row in filtered_gucode_df.iterrows()}
print(new_code_dic)
print('서울특별시의 자치구 수 :', len(new_code_dic))

{'종로구': 1111000000, '중구': 1114000000, '용산구': 1117000000, '성동구': 1120000000, '광진구': 1121500000, '동대문구': 1123000000, '중랑구': 1126000000, '성북구': 1129000000, '강북구': 1130500000, '도봉구': 1132000000, '노원구': 1135000000, '은평구': 1138000000, '서대문구': 1141000000, '마포구': 1144000000, '양천구': 1147000000, '강서구': 1150000000, '구로구': 1153000000, '금천구': 1154500000, '영등포구': 1156000000, '동작구': 1159000000, '관악구': 1162000000, '서초구': 1165000000, '강남구': 1168000000, '송파구': 1171000000, '강동구': 1174000000}
서울특별시의 자치구 수 : 25


## 행정동단위 10개 관심집단수 전처리

### 데이터 읽어오기

In [5]:
file_list = glob.glob('../dedev_2nd_project/23_interest_lifedata/*.xlsx')
all_interest_life_df = []

for m, file_path in enumerate(file_list):
    df = pd.read_excel(file_path)
    if m + 1 >= 10:
        df['month'] = str(m+1)
    else:
        df['month'] = '0' + str(m+1)
    all_interest_life_df.append(df)

interest_life_df = pd.concat(all_interest_life_df, ignore_index=True)

### 1. 법정동코드 최신 코드로 변경
    - 코드값 변경
    - 자치구, 행정동명 컬럼 삭제

In [6]:
for key, val in new_code_dic.items():
    interest_life_df.loc[interest_life_df['자치구'] == key, '행정동코드'] = val

In [7]:
del_interest_life_df = interest_life_df.drop(labels=['자치구', '행정동명'], axis=1)
del_interest_life_df.head()

,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체),month
0,1111000000,1,20,221.44,53.98,7.97,2.22,5.90,3.48,0.00,4.88,7.72,2.32,1.80,0.00,01
1,1111000000,1,25,321.00,107.77,7.85,8.53,12.08,11.15,7.07,10.78,10.63,16.32,16.93,2.34,01
2,1111000000,1,30,349.00,117.23,17.72,12.49,10.43,12.13,7.40,18.66,13.35,15.36,9.66,8.80,01
3,1111000000,1,35,353.00,110.58,21.52,14.27,9.60,12.93,10.15,17.26,21.35,10.42,13.39,6.71,01
4,1111000000,1,40,420.00,79.58,16.60,11.60,10.83,10.16,22.11,11.11,13.40,7.52,6.29,20.09,01


### 2. groupby
    - 행정동코드, 성별, 연령대, month

In [8]:
group_df = del_interest_life_df.groupby(['month', '행정동코드', '성별', '연령대']).mean().reset_index()
group_df.head()

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,1,20,400.602941,147.952941,14.770000,21.176471,19.663529,22.458235,9.571176,8.702353,23.735882,7.662941,13.062941,8.820000
1,01,1111000000,1,25,436.470588,166.825294,17.907059,24.706471,19.280000,27.707647,14.487059,12.697647,26.807647,19.777647,20.644706,15.061765
2,01,1111000000,1,30,381.941176,130.658235,14.430588,16.145294,14.088824,14.951765,10.317059,20.340588,14.840588,16.298824,13.304118,10.086471
3,01,1111000000,1,35,324.647059,98.211176,15.951176,16.204706,8.982941,12.897059,11.767647,13.867647,14.398824,9.776471,9.036471,12.626471
4,01,1111000000,1,40,379.058824,74.508235,13.492353,12.429412,9.315294,9.811176,15.871176,8.663529,11.923529,7.371765,7.480000,15.266471


### 3. 성별
    - 남자(1) -> 0
    - 여자(2) -> 1

In [9]:
group_df['성별'] = group_df['성별'].replace({1:0, 2:1})
group_df.head()

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,400.602941,147.952941,14.770000,21.176471,19.663529,22.458235,9.571176,8.702353,23.735882,7.662941,13.062941,8.820000
1,01,1111000000,0,25,436.470588,166.825294,17.907059,24.706471,19.280000,27.707647,14.487059,12.697647,26.807647,19.777647,20.644706,15.061765
2,01,1111000000,0,30,381.941176,130.658235,14.430588,16.145294,14.088824,14.951765,10.317059,20.340588,14.840588,16.298824,13.304118,10.086471
3,01,1111000000,0,35,324.647059,98.211176,15.951176,16.204706,8.982941,12.897059,11.767647,13.867647,14.398824,9.776471,9.036471,12.626471
4,01,1111000000,0,40,379.058824,74.508235,13.492353,12.429412,9.315294,9.811176,15.871176,8.663529,11.923529,7.371765,7.480000,15.266471


### 4. 연령대 : 20~60대(10대 간격)
    - 연령대 컬럼의 값이 일의자리가 5인 것을 0으로 변경
    - groupby : month, 행정동코드, 성별, 연령대가 같은 것을 Sum
    - 70대 삭제

In [10]:
group_df['연령대'] = group_df['연령대'] // 10 * 10
age_group_df = group_df.groupby(['month', '행정동코드', '성별', '연령대']).sum().reset_index()
age_group_df.head()

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,837.073529,314.778235,32.677059,45.882941,38.943529,50.165882,24.058235,21.400000,50.543529,27.440588,33.707647,23.881765
1,01,1111000000,0,30,706.588235,228.869412,30.381765,32.350000,23.071765,27.848824,22.084706,34.208235,29.239412,26.075294,22.340588,22.712941
2,01,1111000000,0,40,764.352941,145.564118,26.048235,23.829412,16.075294,20.262941,30.027647,16.357647,23.614118,14.064706,12.455882,29.552353
3,01,1111000000,0,50,759.117647,120.308235,20.203529,20.000588,13.432941,19.011765,28.939412,6.760588,19.365294,15.896471,13.487647,31.052941
4,01,1111000000,0,60,464.117647,97.362353,13.312941,13.334118,12.091176,12.105882,13.181765,2.572353,12.244706,10.191176,7.618824,13.964118


In [11]:
del_age_df = age_group_df[age_group_df['연령대'] != 70]
del_age_df.head(10)

,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,837.073529,314.778235,32.677059,45.882941,38.943529,50.165882,24.058235,21.400000,50.543529,27.440588,33.707647,23.881765
1,01,1111000000,0,30,706.588235,228.869412,30.381765,32.350000,23.071765,27.848824,22.084706,34.208235,29.239412,26.075294,22.340588,22.712941
2,01,1111000000,0,40,764.352941,145.564118,26.048235,23.829412,16.075294,20.262941,30.027647,16.357647,23.614118,14.064706,12.455882,29.552353
3,01,1111000000,0,50,759.117647,120.308235,20.203529,20.000588,13.432941,19.011765,28.939412,6.760588,19.365294,15.896471,13.487647,31.052941
4,01,1111000000,0,60,464.117647,97.362353,13.312941,13.334118,12.091176,12.105882,13.181765,2.572353,12.244706,10.191176,7.618824,13.964118
6,01,1111000000,1,20,840.823529,334.481176,25.836471,35.631176,35.296471,37.375294,13.171176,23.877647,30.837647,28.101765,23.932353,13.205882
7,01,1111000000,1,30,662.477059,193.851176,17.954118,25.671765,14.974706,22.251176,14.568235,25.187059,21.875294,18.115882,15.804118,16.305294
8,01,1111000000,1,40,787.705882,134.480588,11.697059,22.268235,9.934118,17.168824,12.710588,12.265294,16.248235,13.647647,11.858235,16.767059
9,01,1111000000,1,50,782.588235,117.028235,11.644118,19.337059,7.910000,19.535882,17.451176,5.141176,17.707059,12.444706,11.998824,25.374118
10,01,1111000000,1,60,515.117647,106.455294,10.242941,18.843529,9.192941,16.066471,14.699412,1.935294,14.920588,10.020000,8.682941,18.590588


### 5. 총인구수, 1인가구수
    - 소수부분 버림

In [12]:
del_age_df['총인구'] = del_age_df['총인구'].astype(int)
del_age_df['1인가구수'] = del_age_df['1인가구수'].astype(int)
del_age_df.iloc[:, 6:] = del_age_df.iloc[:, 6:].round(2)
final_interest_life_df = del_age_df
final_interest_life_df.head()

/tmp/ipykernel_67146/3830424636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_df['총인구'] = del_age_df['총인구'].astype(int)
/tmp/ipykernel_67146/3830424636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_df['1인가구수'] = del_age_df['1인가구수'].astype(int)


,month,행정동코드,성별,연령대,총인구,1인가구수,커뮤니케이션이 적은 집단,평일 외출이 적은 집단,휴일 외출이 적은 집단,출근소요시간 및 근무시간이 많은 집단,외출이 매우 적은 집단(전체),외출이 매우 많은 집단,동영상서비스 이용이 많은 집단,생활서비스 이용이 많은 집단,재정상태에 대한 관심집단,외출-커뮤니케이션이 모두 적은 집단(전체)
0,01,1111000000,0,20,837,314,32.68,45.88,38.94,50.17,24.06,21.40,50.54,27.44,33.71,23.88
1,01,1111000000,0,30,706,228,30.38,32.35,23.07,27.85,22.08,34.21,29.24,26.08,22.34,22.71
2,01,1111000000,0,40,764,145,26.05,23.83,16.08,20.26,30.03,16.36,23.61,14.06,12.46,29.55
3,01,1111000000,0,50,759,120,20.20,20.00,13.43,19.01,28.94,6.76,19.37,15.90,13.49,31.05
4,01,1111000000,0,60,464,97,13.31,13.33,12.09,12.11,13.18,2.57,12.24,10.19,7.62,13.96


In [13]:
final_interest_life_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2250 entries, 0 to 2698
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   month                    2250 non-null   object 
 1   행정동코드                    2250 non-null   int64  
 2   성별                       2250 non-null   int64  
 3   연령대                      2250 non-null   int64  
 4   총인구                      2250 non-null   int64  
 5   1인가구수                    2250 non-null   int64  
 6   커뮤니케이션이 적은 집단            2250 non-null   float64
 7   평일 외출이 적은 집단             2250 non-null   float64
 8   휴일 외출이 적은 집단             2250 non-null   float64
 9   출근소요시간 및 근무시간이 많은 집단     2250 non-null   float64
 10  외출이 매우 적은 집단(전체)         2250 non-null   float64
 11  외출이 매우 많은 집단             2250 non-null   float64
 12  동영상서비스 이용이 많은 집단         2250 non-null   float64
 13  생활서비스 이용이 많은 집단          2250 non-null   float64
 14  재정상태에 대한 관심집단            2250

### CSV로 저장

In [14]:
final_interest_life_df.to_csv('./pre_data/pre_interest_life.csv', index=False)